# Prepare the conda environment

## Install the mini-conda and use it to install diffpy-cmi

In [ ]:
!echo $PYTHONPATH

In [ ]:
%env PYTHONPATH=

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!which conda

In [ ]:
!conda --version

In [ ]:
!conda create -n diffpy -c defaults -c diffpy python=3.7 diffpy-cmi scikit-learn scipy pandas numpy --yes

In [ ]:
!conda env list

## Configure the python to recognize the diffpy library

In [ ]:
!ls /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy*

In [ ]:
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.structure-3.0.1-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.utils-3.0.0-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/

In [ ]:
import sys
sys.path.insert(1, "/usr/local/envs/diffpy/lib/python3.7/site-packages")

## Test if we can import diffpy

In [ ]:
import diffpy.srfit
import diffpy.srreal
import diffpy.structure
import diffpy.utils

In [ ]:
!conda env list

## Configure the python to recognize the diffpy library

In [ ]:
!ls /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy*

In [ ]:
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.structure-3.0.1-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.utils-3.0.0-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/

In [ ]:
import sys
sys.path.insert(1, "/usr/local/envs/diffpy/lib/python3.7/site-packages")

## Test if we can import diffpy

In [ ]:
import diffpy.srfit
import diffpy.srreal
import diffpy.structure
import diffpy.utils

## Download the data

In [ ]:
!git clone https://github.com/st3107/19sab_NMF_PCA_data.git

In [ ]:
!cp -r 19sab_NMF_PCA_data/* /content/

# Demo of NMF

In [ ]:
%matplotlib inline

import os
import string
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from itertools import combinations
from functools import partial

import scipy
from diffpy.structure import loadStructure
from diffpy.srreal.pdfcalculator import PDFCalculator, DebyePDFCalculator
from diffpy.srfit.pdf.characteristicfunctions import sphericalCF

from sklearn.decomposition import PCA, NMF

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['figure.dpi'] = 120

In [ ]:
def norm_ar(ar):
    return (ar - ar.min()) / (ar.max() - ar.min())

In [ ]:
cfg = dict(qmin=0.1, qmax=25, rmin=1.5, rmax=20, qdamp=0.1, qbroad=0.01)
cal = PDFCalculator(**cfg)
stru_ars = []
stru_ars += [loadStructure('RuO2_tetragonal.cif')]
#stru_ars += [loadStructure('LiO2.cif')]
stru_ars += [loadStructure('LiRuO2_mp-28254_conventional_standard.cif')]
#stru_ars += [loadStructure('LiRuO2_icsd.cif')]
stru_ars += [loadStructure('Ru_mp-8639_conventional_standard.cif')]

In [ ]:
# check Uiso and fix if needed
for s in stru_ars:
    print(s.anisotropy, s.Uisoequiv, s.Bisoequiv)

In [ ]:
#d = {k:s.Uisoequiv for k, s in zip(['RuO2_Uiso', 'LiO2_Uiso', 'Ru_Uiso'], stru_ars)}
d = {}
d.update(cfg)
param_df = pd.DataFrame([d])
param_df.to_csv('sim_exp_param.csv')

param_df

In [ ]:
# sanity check on the cif selected

stru_ars[1].Uisoequiv = 0.005
#stru_ars[1].Uisoequiv *= 2

stru_ars[-1].Uisoequiv = 0.004
for s in stru_ars:
    s.Uisoequiv *= 1.5
#stru_ars[1].Uisoequiv *= 20

In [ ]:
for s in stru_ars:
    print(s.Uisoequiv)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8), sharex=True)
grs = []
shift = 16
for i, s in enumerate(stru_ars):
    print(s.Uisoequiv)
    r, g = cal(s)
    if i == 2:
        # add spherical form factor to NP
        g *= sphericalCF(r, 15)
    #g = norm_ar(g)
    #shit = 1
    ax.plot(r, g + i * shift)
    grs += [g]
ax.set_xlabel(r'r ($\mathrm{\AA}$)')
ax.set_ylabel(r'G ($\mathrm{\AA}^{-2}$)')
#fig.savefig('Input_phases.pdf')
#fig.savefig('Input_phases.png')

# Compute angle between input Grs

In [ ]:
def angle_between_(v1, v2):
    val = np.dot(v1, v2) / np.linalg.norm(v1) / np.linalg.norm(v2)
    return np.arccos(np.clip(val, -1, 1))

def angle_between(v1, v2):
    val = np.dot(v1, v2) / np.linalg.norm(v1) / np.linalg.norm(v2)
    return val

In [ ]:
citer = combinations(grs, 2)
for c in citer:
    print(angle_between(*c))

In [ ]:
def linear_rep_cost(params, p1, p2, p_target=None):
    c1, c2 = params
    if p_target:
        return (p1 * c1 + p2 * c2) - p_target
    else:
        return (p1 * c1 + p2 * c2)

# generate ground truth of phase fraction

In [ ]:
def logistic_func(x):
    return np.exp(x) / (1 + np.exp(x))

In [ ]:
unit, grid = 100, 50
x = np.linspace(0.5, unit, grid)
# initial phase -> second order so agressive drop
t0 = (0.5 / 6  * unit)
p0 = logistic_func(-(x - t0))

# 2nd phase -> compliment with initial phase 
p1 = np.zeros_like(p0)
p1 = 1- p0
mask = x >= 1.2 * t0
sig = unit * 0.4
p1[mask] *= np.exp(-(x[mask]/sig)**4)

# 3rd phase -> first order transition 
p2 = np.zeros_like(p1)
p2[mask] = 1 - p0[mask] - p1[mask]
p2 = 1 - p0 - p1

fig, ax = plt.subplots()
ax.plot(x, p0, '--s')
ax.plot(x, p1, '--s')
ax.plot(x, p2, '--s')

sim_phase = (p0, p1, p2)
# sanity check line
#ax.plot(x, p0 + p1 + p2)
#ax.axvline(t0, ls='--', color='k', alpha=0.5)

ax.set_xlabel('Reaction time (a.u.)')
ax.set_ylabel('Phase ratio')
#fig.savefig('synthetic_phase_ratio.pdf')
#fig.savefig('synthetic_phase_ratio.png')

# Generate mixed Gr

In [ ]:
# mixed ar
mixed_ar = None
for (g, p) in zip(grs, [p0, p1, p2]):
    gr_frac = np.outer(g, p) 
    if mixed_ar is None:
        mixed_ar = np.zeros_like(gr_frac)
    mixed_ar += gr_frac
mixed_ar = mixed_ar.T
print(mixed_ar.shape)

In [ ]:
np.save('sim_raw_mixed_Gr', mixed_ar)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 15), dpi=120)
shift = 5
for i, ar in enumerate(mixed_ar[::2]):
    ax.plot(r, ar + i * shift, c='C0', lw=2.5)
ax.set_xlabel(r'r ($\mathrm{\AA}$)')
ax.set_ylabel(r'G ($\mathrm{\AA}^{-2}$)')
ax.text(0.88, 0.95, '(g)',
        transform=ax.transAxes,
        size=18,)            
fig.savefig('Simulated_mixed_Grs.pdf')
fig.savefig('Simulated_mixed_Grs.png')

# PCA and see the result

In [ ]:
pca = PCA(n_components=0.99, random_state=23)
pca.fit(mixed_ar)

print(pca.explained_variance_ratio_)

fig, ax = plt.subplots(figsize=(6, 8))
#fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 7))
shift = 0.1
for i, el in enumerate(pca.components_):
    ax.plot(r, el + i * shift)
ax.set_xlabel(r'r ($\mathrm{\AA}$)', size=18)
ax.set_ylabel(r'$G^e$', size=18)
fig.savefig('PCA_phases.pdf')
fig.savefig('PCA_phases.png')

#for i, el in enumerate(grs):
#    ax1.plot(r, el + i * shift * 100)    
#i += 1
#ax.plot(r, mixed_ar.mean(0)/50 + i * shift)


pca_weight = pca.transform(mixed_ar).T
pca_weight /= 100
#weight /= np.abs(weight).sum(1)[:, np.newaxis]
fig, ax = plt.subplots()
#fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 7), sharey=True)
for el in pca_weight:
    ax.plot(el, '--s')
ax.set_xlabel('Reaction time (a.u.)')
ax.set_ylabel('Phase ratio')
#fig.savefig('PCA_weights.pdf')
#fig.savefig('PCA_weights.png')
#for p in [p0, p1, p2]:
#    ax1.plot(p)

In [ ]:
m = PCA(n_components=20, random_state=23)
m.fit(mixed_ar)
print(m.explained_variance_ratio_)
fig, ax = plt.subplots(figsize=(6, 7))
ax.plot(range(1, len(m.explained_variance_ratio_)+1),
        m.explained_variance_ratio_, '--s', ms=10, mfc='C1')

In [ ]:
#%timeit m.fit(mixed_ar)

# sanity check, reconstructed arrays

In [ ]:
a = pca.transform(mixed_ar)
b = pca.inverse_transform(a)

fig, (ax, ax2) = plt.subplots(2, 1, figsize=(6, 9))
ax.plot(r, b[-1, :])
ax2.plot(r, grs[-1])

# NMF and see the result

In [ ]:
def nmf_shift(ars):
    return ars - ars.flatten().min()

In [ ]:
nmf_ar = nmf_shift(mixed_ar)
fig, ax = plt.subplots(figsize=(6, 9), dpi=120)
shift = 5
for i, ar in enumerate(nmf_ar[::2]):
    ax.plot(r, ar + i * shift, c='C0', lw=1.8)

nmf_loss = []
pca_loss = []
sweeping_grid = range(1, 5 + 1, 1)
for i in sweeping_grid: 
    m = NMF(n_components=i, random_state=23, max_iter=1000)
    mm = PCA(n_components=i, random_state=23)
    m.fit(nmf_ar)
    mm.fit(mixed_ar)
    nmf_loss += [m.reconstruction_err_]
    recon = mm.inverse_transform(mm.transform(mixed_ar))
    pca_loss += [np.square(mixed_ar - recon).sum()]
pca_loss = np.asarray(pca_loss)
nmf_loss = np.asarray(nmf_loss)

In [ ]:
# show loss    
fig, (ax, ax2) = plt.subplots(2, 1, figsize=(6, 9), sharex=True)
plt.subplots_adjust(hspace=0)
ax.plot(sweeping_grid, 1/100 * pca_loss, '--s', ms=10, mfc='C1')
ax2.plot(sweeping_grid, nmf_loss, '--s', ms=10, mfc='C1')
ax2.set_xlabel('Number of components')
ax.set_ylabel('Reconstruction error')
ax2.set_ylabel('Reconstruction error')
#fig.savefig('PCA_NMF_reconstruction_loss.png')
#fig.savefig('PCA_NMF_reconstruction_loss.pdf')

In [ ]:
def nmf_ncomp_selection(loss, rtol=1e-3):
    loss = np.asarray(loss)  # cast anyway
    assert loss.ndim == 1
    # find improvement ratio after adding subsequent comp
    imp_ratio = np.abs(np.diff(loss) / loss[:-1])
    inds, = np.where(imp_ratio <= rtol)
    return inds[0] + 1

In [ ]:
n_comp = nmf_ncomp_selection(nmf_loss)
print(n_comp)
nmf = NMF(n_components=n_comp, random_state=23, max_iter=1000)
nmf.fit(nmf_ar)

In [ ]:
#%timeit m.fit(nmf_ar)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8))
shift = 0
# seq to align with input phase
seq = [2, 1, 0]
for i, s in enumerate(seq):
    ax.plot(r, nmf.components_[s, :] + shift)
    shift += nmf.components_[s, :].max() * 1.1
ax.set_xlabel(r'r ($\mathrm{\AA}$)')
ax.set_ylabel(r'$G^e$')
fig.savefig('NMF_phases.pdf')
fig.savefig('NMF_phases.png')

nmf_weight = nmf.transform(nmf_ar)
nmf_weight /= nmf_weight.sum(1)[:, np.newaxis]
nmf_weight = nmf_weight.T
nmf_weight = np.asarray([nmf_weight[s, :] for s in seq])
fig, ax = plt.subplots()
for w in nmf_weight:
    ax.plot(w, '--s')
ax.set_xlabel('Reaction time (a.u.)')
ax.set_ylabel('Phase ratio')
#fig.savefig('NMF_weights.pdf')
fig.savefig('NMF_weights.png')

# simulating time series plot

In [ ]:
def swipping_nmf_loss(ar, comp_ub=5):
    loss = []
    sweeping_grid = range(1, comp_ub + 1, 1)
    for i in sweeping_grid: 
        m = NMF(n_components=i, random_state=23, max_iter=10000)
        m.fit(ar)
        loss += [m.reconstruction_err_]
    loss = np.asarray(loss)
    return loss

def nmf_ncomp_selection(loss, rtol=1e-3):
    loss = np.asarray(loss)  # cast anyway
    assert loss.ndim == 1
    # find improvement ratio after adding subsequent comp
    imp_ratio = np.abs(np.diff(loss) / loss[:-1])
    inds, = np.where(imp_ratio <= rtol)
    return inds[0] + 1 if inds.size > 0 else 1

def decomp_opt(ar, model):
    model.fit(ar)
    comps = model.components_
    ws = model.transform(ar)
    #ws /= ws.sum(1)[:, np.newaxis]
    return comps, ws.T

In [ ]:
# test drive
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(8, 6))
plt.subplots_adjust(wspace=0.1)
shift = 0
comps, ws = decomp_opt(nmf_ar, nmf)
ws /= ws.sum(0)[np.newaxis, :]
for i, (el, w) in enumerate(zip(comps, ws)):
    ######################
    ax.plot(r, el + shift, c=f'C{i}')
    ax.set_ylabel(r'G$^e$')
    ax.set_xlabel(r'r ($\mathrm{\AA}$)')
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    shift += el.max() * 1.2
    #######################
    ax2.plot(w, '--s', c=f'C{i}', markevery=2)
    ax2.set_xlabel('Discharge time')
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Phase ratio')

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(8, 6))
plt.subplots_adjust(wspace=0.1)
shift = 0
comps, ws = decomp_opt(mixed_ar, pca)
ws /= 100
for i, (el, w) in enumerate(zip(comps, ws)):
    ######################
    ax.plot(r, el + shift, c=f'C{i}')
    ax.set_ylabel(r'G$^e$')
    ax.set_xlabel(r'r ($\mathrm{\AA}$)')
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    shift += el.max() * 1.2
    #######################
    ax2.plot(w, '--s', c=f'C{i}', markevery=2)
    ax2.set_xlabel('Discharge time')
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Phase ratio')

In [ ]:
N = len(nmf_ar)
nmf_recon_loss, nmf_comp_num = [], []
for ii in range(1, N, 1):
    # decomp operation
    ar = nmf_ar[:ii]
    loss = swipping_nmf_loss(ar, comp_ub=5)
    n_comp = nmf_ncomp_selection(loss, rtol=1e-3)
    print(f"INFO: at {ii}, n_comp = {n_comp}")
    decomp_model = NMF(n_components=n_comp, random_state=23, tol=1e-5, max_iter=int(1e4))
    comps, ws = decomp_opt(ar, decomp_model)
    ws /= ws.sum(0)[np.newaxis, :]
    nmf_recon_loss += [decomp_model.reconstruction_err_]
    nmf_comp_num += [n_comp]
    """
    fig, (ax, ax2) = plt.subplots(1, 2, figsize=(8, 6))
    plt.subplots_adjust(wspace=0.1)
    shift = 0
    for i, (el, w) in enumerate(zip(comps, ws)):
        ######################
        ax.plot(r, el + shift, c=f'C{i}')
        ax.set_ylabel(r'G$^e$')
        ax.set_xlabel(r'r ($\mathrm{\AA}$)')
        ax.xaxis.set_major_locator(plt.MultipleLocator(5))
        shift += el.max() * 1.2
        #######################
        ax2.plot(w, '--s', c=f'C{i}', markevery=2)
        ax2.set_xlabel('Discharge time')
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.tick_right()
        ax2.set_ylabel('Phase ratio')
    fig.savefig(f'nmf_video_source/nmf_anime_to{ii}.png', dpi=80)
    plt.close()
    """

In [ ]:
import string

string_iter = iter(string.ascii_lowercase)

fig, axs = plt.subplots(2, 1, figsize=(6, 9), sharex=True)
plt.subplots_adjust(hspace=0)
ms = 4.5
for i, (ax, ar) in enumerate(zip(axs, [nmf_recon_loss, nmf_comp_num])):
    ax.plot(ar, '-s', ls='-', mfc='C6', ms=ms)
    s = next(string_iter)
    ax.text(0.88, 0.87, '({})'.format(s),
                    transform=ax.transAxes,
                    size=16,)            
    #ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    if i == 1:
        ax.yaxis.set_major_locator(plt.MultipleLocator(1))
axs[0].set_ylabel('Reconstruction Error')
axs[1].set_ylabel('Number of Components')
ax.set_xlabel('Discharge time')

axins = ax.inset_axes(bounds=[0.4, 0.15, 0.55, 0.7], transform=ax.transAxes)
for x in sim_phase:
    axins.plot(x, '--s', ms=ms-1.5)
axins.xaxis.set_major_locator(plt.MultipleLocator(10))
axins.yaxis.set_major_locator(plt.MultipleLocator(0.5))
axins.set_ylabel('Phase Ratio', size=15)
axins.tick_params(labelsize=13)
#fig.savefig('nmf_recon_error_num_comp.pdf')

In [ ]:
import string

string_iter = iter(string.ascii_lowercase)

fig, axs = plt.subplots(2, 1, figsize=(6, 9), sharex=True)
plt.subplots_adjust(hspace=0)
ms = 4.5
for i, (ax, ar) in enumerate(zip(axs, [nmf_recon_loss, nmf_comp_num])):
    ax.plot(ar, '-s', ls='-', mfc='C6', ms=ms)
    s = next(string_iter)
    ax.text(0.88, 0.87, '({})'.format(s),
                    transform=ax.transAxes,
                    size=16,)            
    #ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    if i == 1:
        ax.yaxis.set_major_locator(plt.MultipleLocator(1))
axs[0].set_ylabel('Reconstruction Error')
axs[1].set_ylabel('Number of Components')
ax.set_xlabel('Discharge time')

axins = ax.inset_axes(bounds=[0.4, 0.15, 0.55, 0.7], transform=ax.transAxes)
for x in sim_phase:
    axins.plot(x, '--s', ms=ms-1.5)
axins.xaxis.set_major_locator(plt.MultipleLocator(10))
axins.yaxis.set_major_locator(plt.MultipleLocator(0.5))
axins.set_ylabel('Phase Ratio', size=15)
axins.tick_params(labelsize=13)
fig.savefig('nmf_recon_error_num_comp.pdf')

# Static plot

In [ ]:
fig = plt.figure(figsize=(10, 15), dpi=120)
outer_grid = plt.GridSpec(3, 1, wspace=0, hspace=0.1)
end_ = 5
step = 25
for i in range(3):
    inner_grid = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer_grid[i],
                                                  wspace=0, hspace=0)
    end_ += 0 if i == 0 else step
    print(end_)
    ar = mixed_ar[:end_]
    n_comp = 0.99
    decomp_model = PCA(n_components=n_comp, random_state=23)
    comps, ws = decomp_opt(ar, decomp_model)
    ws /= 100
    ax = fig.add_subplot(inner_grid[0])
    ax2 = fig.add_subplot(inner_grid[1])
    shift = 0
    for j, (el, w) in enumerate(zip(comps, ws)):
        ######################
        ax.plot(r, el + shift, c=f'C{j}', lw=2.5)
        ax.xaxis.set_major_locator(plt.MultipleLocator(5))
        ax.yaxis.set_major_locator(plt.MaxNLocator(3))
        shift += el.max() * 1.2
        if i == 1:
            ax.set_ylabel(r'G$^e$', size=18)
        #######################
        sep = 1 if end_ < 20 else 2
        ax2.plot(w, '--s', c=f'C{j}', markevery=sep)
        #ax2.set_xlabel('Discharge time')
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.tick_right()
        ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
        if i == 1:
            ax2.set_ylabel('Phase ratio', size=18)
ax2.set_xlabel('Discharge time', size=18)
ax.set_xlabel(r'r ($\mathrm{\AA}$)', size=18)
fig.savefig('static_pca_decom_process.pdf')

In [ ]:
fig = plt.figure(figsize=(10, 15), dpi=120)
outer_grid = plt.GridSpec(3, 1, wspace=0, hspace=0.1)
end_ = 5
step = 25
for i in range(3):
    inner_grid = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer_grid[i],
                                                  wspace=0, hspace=0)
    end_ += 0 if i == 0 else step
    print(end_)
    ar = nmf_ar[:end_]
    loss = swipping_nmf_loss(ar, comp_ub=5)
    n_comp = nmf_ncomp_selection(loss, rtol=1e-3)
    decomp_model = NMF(n_components=n_comp, random_state=23, max_iter=1500)
    comps, ws = decomp_opt(ar, decomp_model)
    ws /= ws.sum(0)[np.newaxis, :]
    ax = fig.add_subplot(inner_grid[0])
    ax2 = fig.add_subplot(inner_grid[1])
    shift = 0
    for j, (el, w) in enumerate(zip(comps, ws)):
        ######################
        ax.plot(r, el + shift, c=f'C{j}', lw=2.5)
        ax.xaxis.set_major_locator(plt.MultipleLocator(5))
        ax.yaxis.set_major_locator(plt.MaxNLocator(3))
        shift += el.max() * 1.2
        if i == 1:
            ax.set_ylabel(r'G$^e$', size=18)
        #######################
        sep = 1 if end_ < 20 else 2
        ax2.plot(w, '--s', c=f'C{j}', markevery=sep)
        #ax2.set_xlabel('Discharge time')
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.tick_right()
        ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
        if i == 1:
            ax2.set_ylabel('Phase ratio', size=18)
ax2.set_xlabel('Discharge time', size=18)
ax.set_xlabel(r'r ($\mathrm{\AA}$)', size=18)
fig.savefig('static_nmf_decom_process.pdf')

# Big-fat-ass plot

In [ ]:
fig = plt.figure(figsize=(18, 15), dpi=120)
outer_grid = plt.GridSpec(3, 2, wspace=0.15, hspace=0.1)
end_ = 5
step = 25
string_iter = iter(string.ascii_lowercase)
label_size = 28

for i in range(0, 6, 2):
    inner_grid = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer_grid[i],
                                                  wspace=0, hspace=0)
    end_ += 0 if i == 0 else step
    print(end_)
    ar = mixed_ar[:end_]
    n_comp = 0.99
    decomp_model = PCA(n_components=n_comp, random_state=23)
    comps, ws = decomp_opt(ar, decomp_model)
    ws /= 100
    ax = fig.add_subplot(inner_grid[0])
    ax2 = fig.add_subplot(inner_grid[1])
    shift = 0
    for j, (el, w) in enumerate(zip(comps, ws)):
        ######################
        el *= 10
        ax.plot(r, el + shift, c=f'C{j}', lw=2.5)
        ax.xaxis.set_major_locator(plt.MultipleLocator(5))
        ax.yaxis.set_major_locator(plt.MaxNLocator(3))
 
        shift += el.max() * 1.5
        if i == 2:
            ax.set_ylabel(r'G$^e$', size=label_size + 2)
        #######################
        sep = 1 if end_ < 20 else 2
        ax2.plot(w, '--s', c=f'C{j}', markevery=sep)
        #ax2.set_xlabel('Discharge time')
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.tick_right()
        ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
#        if i == 2:
#            ax2.set_ylabel('Phase ratio', size=18)
    s = next(string_iter)
    ax.text(0.88, 0.88, '({})'.format(s),
                    transform=ax.transAxes,
                    size=16,)            
    s = next(string_iter)
    xloc = 0.83 if i == 0 else 0.86
    ax2.text(xloc, 0.88, '({})'.format(s),
            transform=ax2.transAxes,
            size=16,
            )
ax2.set_xlabel('Discharge time', size=label_size)
ax.set_xlabel(r'r ($\mathrm{\AA}$)', size=label_size)
            
end_ = 5
for i in range(1, 7, 2):
    inner_grid = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer_grid[i],
                                                  wspace=0, hspace=0)
    end_ += 0 if i == 1 else step
    print(end_)
    ar = nmf_ar[:end_]
    loss = swipping_nmf_loss(ar, comp_ub=5)
    n_comp = nmf_ncomp_selection(loss, rtol=1e-3)
    decomp_model = NMF(n_components=n_comp, random_state=23, max_iter=1500)
    comps, ws = decomp_opt(ar, decomp_model)
    ws /= ws.sum(0)[np.newaxis, :]
    ax = fig.add_subplot(inner_grid[0])
    ax2 = fig.add_subplot(inner_grid[1])
    shift = 0
    for j, (el, w) in enumerate(zip(comps, ws)):
        ######################
        ax.plot(r, el + shift, c=f'C{j}', lw=2.5)
        ax.xaxis.set_major_locator(plt.MultipleLocator(5))
        ax.yaxis.set_major_locator(plt.MaxNLocator(3))
        shift += el.max() * 1.1
#        if i == 3:
#            ax.set_ylabel(r'G$^e$', size=18)
        #######################
        sep = 1 if end_ < 20 else 2
        ax2.plot(w, '--s', c=f'C{j}', markevery=sep)
        #ax2.set_xlabel('Discharge time')
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.tick_right()
        ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
        if i == 1 + 2:
            ax2.set_ylabel('Phase ratio', size=label_size + 2)
            
    s = next(string_iter)
    ax.text(0.88, 0.88, '({})'.format(s),
                    transform=ax.transAxes,
                    size=16,)            
    s = next(string_iter)
    xloc = 0.83 if i == 0 else 0.86
    ax2.text(xloc, 0.88, '({})'.format(s),
            transform=ax2.transAxes,
            size=16,
            )
ax2.set_xlabel('Discharge time', size=label_size)
ax.set_xlabel(r'r ($\mathrm{\AA}$)', size=label_size)
fig.savefig('static_pca_nmf_decom_process.pdf')

In [ ]:
def plot_ar(ax, ars, scale):
    shift = 0
    offsets = []
    for i, ar in enumerate(ars):
        ax.plot(r, ar + shift)
        offsets += [float(shift)]
        shift = shift + scale * ar.max()
    return offsets
        
# construct ars to plot
pca_ars = np.asarray([x * 10 for x in pca.components_])
nmf_ars = np.asarray([nmf.components_[i, :] for i in seq])
gr_ars = np.asarray(grs) / 10
print(pca_ars.shape, nmf_ars.shape, gr_ars.shape)

In [ ]:
for a, b in zip(nmf_ars, gr_ars):
    print(np.corrcoef(a, b)[0, 1])

In [ ]:
fig = plt.figure(figsize=(10, 15), dpi=120)
grid = plt.GridSpec(3, 2, fig, hspace=0., wspace=0.0)
N = 6
label_size=19
# first col
plot_args = [(pca_ars, 1.6), (gr_ars, 1.5), (nmf_ars, 1.05)]
plotted_axs = []
sub_axs = []
for i, arg in enumerate(plot_args):
    ax = fig.add_subplot(grid[i, 0], sharey=None)
    offsets = plot_ar(ax, *arg)
    for offset in offsets:
        ax.axhline(offset, ls='--', color='k', alpha=0.5)
    #ax.text(0.1, 0.1, '{}'.format(i), size=20)
    sub_axs.append(ax)
    ax.set_xlim([0, 21])
    #ax.yaxis.set_major_locator(plt.MaxNLocator(3))
for i, el in enumerate(sub_axs):
    if i != 2:
        el.set_xticklabels([])
    if i != 1:
        el.set_ylabel(r'$G^e$', size=label_size)
    else:
        #el.set_ylabel(r'G ($\mathrm{\AA}^{-2}$)', size=label_size)
        el.set_ylabel(r'$G$', size=label_size)
    el.yaxis.set_major_locator(plt.MultipleLocator(1))
    #el.yaxis.set_major_locator(plt.MaxNLocator(3))
sub_axs[-1].set_xlabel(r'r ($\mathrm{\AA}$)', size=label_size)
plotted_axs.extend(sub_axs)

# second col
plot_args = [pca_weight, sim_phase, nmf_weight,]
sub_axs = []
_y = None
for i, ars in enumerate(plot_args):
    ax = fig.add_subplot(grid[i, 1], sharey=_y)
    #_y = ax
    for ar in ars:
        ax.plot(range(50), ar, '--s', markevery=2)
    #if i == 0:
    ax.axhline(0, ls='--', color='k', alpha=0.5)
    #ax.text(0.1, 0.1, 'a{}'.format(i), size=20)
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.5))
    ax.xaxis.set_major_locator(plt.MultipleLocator(10))
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    #ax.set_ylim(top=1.2)
    sub_axs.append(ax)
sub_axs[1].set_ylabel('Phase ratio', size=label_size)
sub_axs[-1].set_xlabel('Discharge time (a.u.)')
plotted_axs.extend(sub_axs)
# label
string_iter = iter(string.ascii_lowercase)
n_row = 3
yloc = [0.9] + [0.88] * 2
for i in range(n_row):
    # first column
    ax = plotted_axs[i]
    s = next(string_iter)
    ax.text(0.88, 0.90, '({})'.format(s),
            transform=ax.transAxes,
            size=16,
           )
    # second column
    ax = plotted_axs[i + n_row]
    s = next(string_iter)
    ax.text(0.88, yloc[i], '({})'.format(s),
            transform=ax.transAxes,
            size=16,
           )
fig.savefig('nmf_pca_cf.pdf')

# Experimenting different phase evolvement for PCA

In [ ]:
unit, grid = 100, 50
x = np.linspace(0.5, unit, grid)
# initial phase -> drop
t0 = (0.5 / 6  * unit)
#p0 = logistic_func(-(x - t0))
sig = unit * 0.35
p0 = np.exp(-((x)/sig)**4)

# 2nd phase -> compliment with initial phase 
p1 = np.zeros_like(p0)
p1 = 1- p0
mask = x >=  0.6 * t0
sig = unit * 0.4
p1[mask] *= np.exp(-(x[mask]/sig)**2)
#sig = unit * 0.1
#p1 = 0.5 * np.exp(-((x - t0)/sig)**2)

# 3rd phase -> first order transition 
p2 = np.zeros_like(p1)
p2[mask] = 1 - p0[mask] - p1[mask]
p2 = 1 - p0 - p1
p2 = p2.clip(0)

fig, ax = plt.subplots()
ax.plot(x, p0, '--s')
ax.plot(x, p1, '--s')
ax.plot(x, p2, '--s')

sim_phase = (p0, p1, p2)
# sanity check line
#ax.axvline(t0, ls='--', color='k', alpha=0.5)

ax.set_xlabel('Reaction time (a.u.)')
ax.set_ylabel('Phase ratio')
fig.savefig('tweak_phase_ratio.pdf', dpi=120)

In [ ]:
# mixed ar
mixed_ar = None
for (g, p) in zip(grs, [p0, p1, p2]):
    gr_frac = np.outer(g, p) 
    if mixed_ar is None:
        mixed_ar = np.zeros_like(gr_frac)
    mixed_ar += gr_frac
mixed_ar = mixed_ar.T
print(mixed_ar.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 9), dpi=120)
shift = 5
for i, ar in enumerate(mixed_ar[::2]):
    ax.plot(r, ar + i * shift, c='C0', lw=1.8)
ax.set_xlabel(r'r ($\mathrm{\AA}$)')
ax.set_ylabel(r'G ($\mathrm{\AA}^{-2}$)')

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(8, 6))
plt.subplots_adjust(wspace=0.1)
shift = 0
model = PCA(n_components=0.99, random_state=23)
comps, ws = decomp_opt(mixed_ar[:], model)
ws /= 100
for i, (el, w) in enumerate(zip(comps, ws)):
    ######################
    ax.plot(r, el + shift, c=f'C{i}')
    ax.set_ylabel(r'G$^e$')
    ax.set_xlabel(r'r ($\mathrm{\AA}$)')
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    shift += el.max() * 1.2
    #######################
    ax2.plot(w, '--s', c=f'C{i}', markevery=2)
    ax2.set_xlabel('Discharge time')
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Phase ratio')
fig.savefig('tweak_PCA_result.pdf', dpi=80)